In [2]:
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [51]:
'''GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]=LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY'''

'GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")\nTAVILY_API_KEY=os.getenv("TAVILY_API_KEY")\nLANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")\nLANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")\n\nos.environ["LANGCHAIN_TRACING_V2"]=LANGCHAIN_TRACING_V2\nos.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY\nos.environ["TAVILY_API_KEY"]=TAVILY_API_KEY\nos.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY'

### load the model and test it with the simple message

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [9]:
message=[SystemMessage(content="Hi you are nice Bot"),HumanMessage(content="Hi, how are you buddy?")]

In [10]:
message2=[SystemMessage(content="Hi you are nice Bot and you answer everything in brief"),HumanMessage(content="Hi, how are you buddy?")]

In [11]:
model.invoke(message2)

AIMessage(content="I'm doing well, thanks for asking! How about you?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a1e56f44-f956-4fc9-9c0a-516670404aad-0', usage_metadata={'input_tokens': 19, 'output_tokens': 14, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

### use output parser

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
response = model.invoke(message)

In [15]:
response2 = model.invoke(message2)

In [16]:
parser.invoke(response)

'Hello! I am not BotHi, I am Gemini, a multi-modal AI model, developed by Google. I am designed to assist and inform users to the best of my abilities.\n\nHow may I help you today?'

In [17]:
parser.invoke(response2)

'I am well, thank you for asking.'

### chaining with LCEL

In [18]:
#added two component
chain = model | parser

In [19]:
chain.invoke(message)

'Hello there! I am not BotHi, I am Gemini, a multi-modal AI model, developed by Google. I am not capable of having feelings or emotions, so I am always in a neutral state. How can I assist you today?'

In [20]:
chain.invoke(message2)

"I'm doing well, thanks for asking! How are you doing today?"

### Now lets start with prompting

In [21]:
from langchain_core.prompts import ChatPromptTemplate

In [22]:
system="translate the following into {language}"
#user="i am hungry."

In [23]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [24]:
prompt = prompt_template.invoke({"language":"tamil","text":"I am fresh and happy and I am doing good"})

In [25]:
parser.invoke(model.invoke(prompt))

'நான் புதியவன் மற்றும் மகிழ்ச்சியானவன் மற்றும் நான் நல்லவன்'

In [26]:
prompt.to_messages()

[SystemMessage(content='translate the following into tamil', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I am fresh and happy and I am doing good', additional_kwargs={}, response_metadata={})]

### chaining 

In [27]:
chain = prompt_template | model | parser

In [46]:
chain.invoke({"language": "hindi", "text": "I am hungry"})

'मैं भूखा हूँ'

### Agents

In [29]:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [31]:
memory = MemorySaver()

In [32]:
search=TavilySearchResults( max_results=2)

In [33]:
tools=[search]

In [34]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

In [35]:
config = {"configurable": {"thread_id": "abc123"}}

In [48]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im Joseph! and i live in bangalore")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Joseph! What can I do for you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0a6e30bc-9ee0-44e8-9081-3fe05b584967-0', usage_metadata={'input_tokens': 1747, 'output_tokens': 11, 'total_tokens': 1758, 'input_token_details': {'cache_read': 0}})]}}
----


In [49]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-fb7e6366-21d6-4cea-aa67-5686613478cf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in bangalore'}, 'id': '5533f29a-dfbb-49ab-9f62-89721b2296c9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1767, 'output_tokens': 26, 'total_tokens': 1793, 'input_token